In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver_manager
!pip install beautifulsoup4

In [11]:
import requests
from bs4 import BeautifulSoup
import json

# 크롤링할 데이터를 저장할 리스트
jobs = []

page = 1  # 페이지 번호 초기값 설정
while True:
    url = f'https://www.peoplenjob.com/jobs?field=all&q=데이터&page={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    rows = soup.find_all('tr')

    # 각 공고에서 필요한 정보 추출 및 리스트에 추가
    for row in rows:
        # 공고 제목
        title_tag = row.find('td', class_='job-title')
        title = title_tag.find('a').text.strip() if title_tag else 'N/A'

        # 회사 이름
        company_tag = row.find('td', class_='name')
        company_name = company_tag.find('a').text.strip() if company_tag else 'N/A'

        # 상세 페이지 URL
        detail_url_tag = title_tag.find('a') if title_tag else None
        detail_url = f"https://www.peoplenjob.com{detail_url_tag['href']}" if detail_url_tag else 'N/A'

        # 마감일
        end_date_tag = row.find('span', class_='job-fin-date')
        end_date = end_date_tag.text.strip() if end_date_tag else 'N/A'

        # 카테고리 (직무 타입)
        category_tag = row.find('td', class_='job_type')
        category_name = category_tag.text.strip() if category_tag else 'N/A'

        # 기술 스택 (이 사이트에는 없으므로 'N/A'로 처리)
        stack = 'N/A'

        # 모든 <td> 태그를 리스트로 가져옴
        td_tags = row.find_all('td')

        # <td> 태그가 최소한 2개 이상 있는지 확인
        if len(td_tags) >= 2:
            region_tag = td_tags[-2].find('a')  # 두 번째 마지막 <td>에서 <a> 태그 찾기
            region = region_tag.text.strip() if region_tag else 'N/A'
        else:
            region = 'N/A'  # <td> 태그가 2개 미만인 경우 N/A로 설정

        # 신입/경력 (이 사이트에는 없으므로 'N/A'로 처리)
        career = 'N/A'

        # 출처 (플랫폼 이름)
        platform_name = 'peoplenjob'

        # 추출한 데이터를 딕셔너리로 추가
        job_data = {
            'title': title,
            'company_name': company_name,
            'detail_url': detail_url,
            'end_date': end_date,
            'platform_name': platform_name,
            'category_name': category_name,
            'stack': stack,
            'region': region,
            'career': career
        }

        # 리스트에 저장
        jobs.append(job_data)

    print(f"{page} 페이지 크롤링 완료")

    # '다음' 버튼이 비활성화된 경우 크롤링 종료
    next_button = soup.find('li', class_='next disabled')
    if next_button:
        print("마지막 페이지에 도달하여 크롤링을 종료합니다.")
        break

    page += 1  # 다음 페이지로 이동

# JSON 파일로 저장
json_file = '/Users/youyoungcheon/Desktop/django-project/crawling/peoplenjob_job_list.json'
with open(json_file, 'w', encoding='utf-8') as f:
    json.dump(jobs, f, ensure_ascii=False, indent=4)

print(f"JSON 파일 저장 완료: '{json_file}'")


1 페이지 크롤링 완료
2 페이지 크롤링 완료
3 페이지 크롤링 완료
4 페이지 크롤링 완료
5 페이지 크롤링 완료
6 페이지 크롤링 완료
7 페이지 크롤링 완료
8 페이지 크롤링 완료
9 페이지 크롤링 완료
10 페이지 크롤링 완료
11 페이지 크롤링 완료
12 페이지 크롤링 완료
13 페이지 크롤링 완료
14 페이지 크롤링 완료
15 페이지 크롤링 완료
16 페이지 크롤링 완료
17 페이지 크롤링 완료
18 페이지 크롤링 완료
19 페이지 크롤링 완료
20 페이지 크롤링 완료
21 페이지 크롤링 완료
22 페이지 크롤링 완료
23 페이지 크롤링 완료
24 페이지 크롤링 완료
25 페이지 크롤링 완료
26 페이지 크롤링 완료
27 페이지 크롤링 완료
마지막 페이지에 도달하여 크롤링을 종료합니다.
JSON 파일 저장 완료: '/Users/youyoungcheon/Desktop/django-project/crawling/peoplenjob_job_list.json'


In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import json

# Selenium WebDriver 설정 (Chrome을 예시로 사용)
options = webdriver.ChromeOptions()
options.add_argument('headless')  # 브라우저 창이 뜨지 않도록
driver = webdriver.Chrome(options=options)

# Remember 앱 URL로 이동
url = 'https://career.rememberapp.co.kr/job/postings?search=%7B%22leaderPosition%22%3Afalse%2C%22organizationType%22%3A%22all%22%2C%22applicationType%22%3A%22all%22%2C%22keywords%22%3A%5B%22데이터%22%5D%7D'
base_url = 'https://career.rememberapp.co.kr/'
driver.get(url)

# 페이지 로딩을 기다림
time.sleep(3)

# 데이터를 담을 리스트
jobs = []

# 스크롤을 끝까지 내리며 데이터를 로드
last_height = driver.execute_script("return document.body.scrollHeight")
scroll_count = 0

while True:
    # 페이지의 HTML을 가져와서 BeautifulSoup으로 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 데이터 추출 (ul 태그 내 li 태그에 각각의 공고가 있다고 가정)
    job_list = soup.find_all('div', class_='sc-f0acadd4-0 bYquQe')  # 적절한 태그와 클래스로 수정해야 함

    for job in job_list:
        # 공고 제목 추출
        title_tag = job.find('h4', class_='sc-f0acadd4-7 hQClbs')
        title = title_tag.text.strip() if title_tag else 'N/A'

        # 회사 이름 추출
        company_tag = job.find('p', class_='sc-f0acadd4-5 gCYsaW')
        company_name = company_tag.text.strip() if company_tag else 'N/A'

        #디테일 페이지로 가는 주소
        detail_url = job.find('a', class_='sc-567718ed-0 jzTeOc')['href'] if job.find('a', class_='sc-567718ed-0 jzTeOc') else ''
        detail_url = base_url + detail_url if detail_url else ''
        
        # 경력과 지역을 구분하여 가져오기
        career_region_tags = job.find_all('p', class_='sc-f0acadd4-10')

        career = 'N/A'
        region = 'N/A'
        for tag in career_region_tags:
            text = tag.text.strip()
            if '년' in text:  # "년"이 들어간 텍스트는 경력 정보
                career = text
            else:  # 나머지는 지역 정보로 간주
                region = text

        # 직무 카테고리 (사이트에 없다면 N/A로 처리)
        category_name = 'N/A'
        
        # 기술 스택 (이 사이트에 해당 정보가 없으므로 'N/A'로 처리)
        stack = 'N/A'
        
        # 출처 (플랫폼 이름)
        platform_name = 'remember'

        # 딕셔너리로 데이터 저장
        jobs.append({
            'title': title,
            'company_name': company_name,
            'detail_url': detail_url,
            'end_date': 'N/A',  # end_date를 찾을 수 없는 경우 'N/A'
            'platform_name': platform_name,
            'category_name': category_name,
            'stack': stack,
            'region': region,
            'career': career
        })

    # 스크롤을 아래로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 새로운 데이터가 로드될 때까지 잠시 대기
    time.sleep(2)

    # 스크롤 후 새 높이를 측정
    new_height = driver.execute_script("return document.body.scrollHeight")

    # 더 이상 로드되는 데이터가 없으면 종료
    if new_height == last_height:
        print("더 이상 스크롤 할 수 없습니다.")
        break

    last_height = new_height
    scroll_count += 1

    # 원하는 횟수만큼 스크롤 후 멈추고 싶다면 조건 추가
    #if scroll_count > 10:  # 예: 10번 스크롤 후 멈추기
        #break
print(scroll_count)
# 브라우저 닫기
driver.quit()

# JSON 파일로 저장
with open('remember_job_list.json', 'w', encoding='utf-8') as f:
    json.dump(jobs, f, ensure_ascii=False, indent=4)

print("JSON 파일 저장 완료: 'remember_job_list.json'")


더 이상 스크롤 할 수 없습니다.
7
JSON 파일 저장 완료: 'remember_job_list.json'
